In [ ]:
import sys
import os
project_root = os.path.dirname(os.path.dirname(os.getcwd()))
sys.path.insert(0, project_root)
#импорты загрузки 

import InterfaceWidgets.data_load_widgets as data_load_widgets
import InterfaceWidgets.column_selection_widgets as column_selection_widgets

#импорты EDA
import InterfaceWidgets.EDA_widgets as eda_widgets

#импорты экспериментального ядра
import ExperimentsCore.experiments_core as exp_core
from ExperimentsCore.experiments_engine import run_experiment
import InterfaceWidgets.experiment_config_widgets as exp_config_widgets
import ipywidgets as widgets
from IPython.display import display, clear_output
import numpy as np
import pandas as pd
import plotly.graph_objects as go

# Импорт визуализации
import DataVisualizations.visualizations as viz
from DataVisualizations.means_vis_bootstrap import visualize_means_distribution

# Создаем виджет Output для вывода информации о выбранной колонке
column_info_output = widgets.Output()

# Создаем виджет Output для информации о создании распределения
distribution_info_output = widgets.Output()

# Создаем виджет Output для эксперимента и его результатов
experiment_output = widgets.Output()

# Переменные для хранения текущих объектов
current_statistics = None
current_rv = None
current_df_dist = None

# Функция для расчета статистик распределения
def calculate_distribution_statistics(rv, df_dist):
    """
    Рассчитывает различные статистики для распределения.
    
    Args:
        rv: Объект дискретного распределения
        df_dist: DataFrame с значениями и вероятностями
        
    Returns:
        dict: Словарь с рассчитанными статистиками
    """
    stats = {}
    
    # Рассчитываем среднее
    if hasattr(rv, 'mean'):
        stats['mean'] = rv.mean()
    else:
        # Если у rv нет метода mean(), вычисляем вручную
        xk = df_dist['value'].values
        pk = df_dist['probability'].values
        stats['mean'] = np.sum(xk * pk)
    
    # Рассчитываем медиану (как квантиль 0.5)
    # Для дискретного распределения найдем значение, где CDF >= 0.5
    try:
        median_idx = np.where(df_dist['cumulative_probability'] >= 0.5)[0][0]
        stats['median'] = df_dist['value'].iloc[median_idx]
    except (IndexError, KeyError):
        stats['median'] = None
    
    # В будущем здесь можно добавить расчет других статистик
    # stats['quantile'] = ...
    # stats['proportion'] = ...
    
    return stats

# Обработчик обновления прогресса эксперимента
def update_experiment_progress(current_size, power, target_power, iteration, total_iterations):
    with experiment_output:
        clear_output(wait=True)
        print(f"Текущий размер выборки: {current_size}")
        print(f"Текущая мощность: {power:.3f}")
        print(f"Целевая мощность: {target_power}")
        print(f"Прогресс: {iteration}/{total_iterations}")

# Обработчик запуска эксперимента на основе конфигурации
def handle_experiment_config(config):
    global current_rv
    
    if current_rv is None:
        with experiment_output:
            clear_output(wait=True)
            print("Ошибка: Не найдено дискретное распределение для эксперимента.")
        return
    
    with experiment_output:
        clear_output(wait=True)
        print("Запуск эксперимента с конфигурацией:")
        print(config)
        
        # Создаем прогресс-бар с помощью ipywidgets
        progress_bar = widgets.FloatProgress(
            value=0,
            min=0,
            max=1.0,
            description='Прогресс:',
            bar_style='info',
            style={'bar_color': '#2C5E99'},
            orientation='horizontal'
        )
        display(progress_bar)
    
    # Запускаем эксперимент
    results = run_experiment(current_rv, config, on_progress_update=update_experiment_progress)
    
    # Визуализируем результаты
    with experiment_output:
        clear_output(wait=True)
        print("Эксперимент завершен!")
        print(f"Результаты для {len(results)} размеров выборки:")
        
        # Создаем график с помощью нашей функции визуализации
        fig = viz.power_analysis_plot(results, target_power=config['target_power'])
        
        # Преобразуем в FigureWidget для отображения в виджете
        fig_data = fig.data
        fig_layout = fig.layout
        figure_widget = go.FigureWidget(data=fig_data, layout=fig_layout)
        
        # Отображаем график
        display(figure_widget)
        
        # Отображаем таблицу с результатами
        display(results)
        
        # Если мы достигли целевой мощности, добавляем визуализацию распределения средних
        if len(results) > 0 and results['power'].max() >= config['target_power']:
            print("\nРаспределение средних для оптимального размера выборки:")
            
            # Находим минимальный размер выборки с достаточной мощностью
            optimal_sample_size = results.loc[results['power'] >= config['target_power'], 'sample_size'].min()
            
            # Визуализируем распределение средних
            means_fig = visualize_means_distribution(
                current_rv, 
                optimal_sample_size, 
                config['mde_percent'],
                statistic=config['statistic']
            )
            
            # Преобразуем в FigureWidget для отображения
            means_fig_widget = go.FigureWidget(data=means_fig.data, layout=means_fig.layout)
            display(means_fig_widget)

# Обработчик создания распределения после EDA
def handle_distribution_creation(series):
    global current_statistics, current_rv, current_df_dist
    
    with distribution_info_output:
        distribution_info_output.clear_output(wait=True)
        print("Создание дискретного распределения...")
        current_rv, current_df_dist, round_digits = exp_core.discrete_dist_creation(series)
        
        # Сохраняем статистики для использования в интерфейсе конфигурации
        current_statistics = calculate_distribution_statistics(current_rv, current_df_dist)
            
        display(current_rv, current_df_dist, round_digits)
    
    # Отображаем интерфейс конфигурации с передачей статистик распределения
    exp_config_widgets.display_experiment_config_interface(
        current_statistics=current_statistics,
        on_config_created=handle_experiment_config
    )
    
    # Отображаем виджет для вывода результатов эксперимента ПОСЛЕ интерфейса конфигурации
    display(experiment_output)

# Обработчик выбора колонки
def handle_column_selection(selected_column, data_dict):
    with column_info_output:
        column_info_output.clear_output(wait=True)
        # Удаляем дублирующееся сообщение о выбранной колонке
        # print(f"Получен словарь данных с {len(data_dict['df'])} строками")
    # Запускаем EDA анализ для выбранной колонки с передачей callback'а для создания распределения
    series = data_dict['df'][selected_column]
    eda_widgets.run_eda_analysis(selected_column, data_dict, 
                                on_eda_complete=lambda: handle_distribution_creation(series))

# Обработчик загрузки данных
def handle_data_loaded(data_dict):
    # Когда данные загружены, показываем селектор колонок
    column_selection_widgets.column_selection_widgets(
        data_dict, 
        on_column_selected=handle_column_selection
    )
    # Отображаем виджет для информации о выбранной колонке после селектора
    display(column_info_output)
    # Виджет для информации о распределении отобразится после графиков в функции run_eda_analysis

# Показываем интерфейс загрузки с callback'ом
data_load_widgets.display_interface(on_data_loaded=handle_data_loaded)